In [ ]:
#Set up
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install beautifulsoup4

In [ ]:
#Define getting news function
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

#Locate the target website
driver.get('https://www.snopes.com/?s=covid')

def get_snopes(link):
    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--headless')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome('chromedriver',options=options)

    wait = WebDriverWait(driver, 10)

    driver.get(link)

    result = driver.find_element_by_id("result-list")

    temp_link = None
    list_rows = []
    for items in result.find_elements_by_class_name('list-group-item'):
        list_cells = []
        for item in items.find_elements_by_class_name('heading'):
            links = item.find_elements_by_tag_name('a')
            if len(links):
              temp_link = links[0].get_attribute("href")
            list_cells.append(item.text)
        if not len(list_cells):
          continue
        for item2 in items.find_elements_by_class_name('subheading'):
            date = item2.find_elements_by_class_name('date')
            list_cells.append(item2.text)
        list_cells.append(temp_link)
        list_rows.append(list_cells)

    driver.quit()
    return list_rows

In [ ]:
#Get all news
all_news = []
for i in range(1,210):
    link = "https://www.snopes.com/page/"+str(i)+"/?s=covid"
    all_news += get_snopes(link)

In [ ]:
len(processed_all_news)

2506

In [ ]:
#Split the date and claim
processed_all_news = []
for i in all_news:
    temp = i
    try:
        idx = temp[1].index("-")
    except:
        continue
    temp.append(temp[1][idx+2:])
    temp[1] = temp[1][:idx-1]
    processed_all_news.append(temp)
processed_all_news

In [ ]:
FactCheckData = []

In [ ]:
#Get all fact check news
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

for x in range(490,len(processed_all_news)):
    print(x)
    driver = webdriver.Chrome('chromedriver',options=options)
    driver.get(processed_all_news[x][2])

    if processed_all_news[x][2].startswith("https://www.snopes.com/fact-check"):
      whole_data = driver.find_element_by_class_name("card-body").find_elements_by_xpath("//div[@class='media-body d-flex flex-column align-self-center']/span")
      print(whole_data[0].text)
      temp_row = processed_all_news[x]
      temp_row.append(whole_data[0].text)
      
      FactCheckData.append(temp_row)

    driver.quit()
FactCheckData

In [ ]:
#Create Data Frame
import pandas as pd
import numpy as np

df = pd.DataFrame(processed_all_news), columns = ['Debunking_sites_title','Date','Debunking_sites','Claim','Veracity'])

df

In [ ]:
temp_df = df.copy()
from datetime import datetime

def mdy_to_ymd(B):
    return datetime.strptime(B, '%d %B %Y').strftime('%Y-%m-%d')

for i,r in temp_df.iterrows():
  temp = r["Date"]
  r["Date"] = mdy_to_ymd(temp)

In [ ]:
#see if dates can be compared by strings
temp_df.iloc[0]["Date"] > "2020-06-30"

True

In [ ]:
# filter date
date_filtered  = []
for i,r in temp_df.iterrows():
  if r["Date"] > "2020-06-30" and r["Date"] < "2021-07-01":
    date_filtered.append(temp_df.iloc[i])

date_filtered = pd.DataFrame(date_filtered, columns=list(temp_df.columns.values))

In [ ]:
date_filtered

,Debunking_sites_title,Date,Debunking_sites,Claim
0,"No, Four British Airways Pilots Did Not Die Fr...",2021-06-22,https://www.snopes.com/fact-check/british-airw...,"Unscrupulous conspiracy theorists cynically, a..."
1,Nearly 4K COVID-19 Patients in Massachusetts H...,2021-06-24,https://www.snopes.com/fact-check/4000-vaccina...,The Centers for Disease Control and Prevention...
2,Nearly All COVID Deaths in US Are Now Among Un...,2021-06-24,https://www.snopes.com/ap/2021/06/24/nearly-al...,Deaths per day — now down to under 300 — could...
3,Almost 900 Secret Service Employees Were Infec...,2021-06-22,https://www.snopes.com/ap/2021/06/22/almost-90...,More than 11% of Secret Service employees were...
4,Palace: Japan Emperor ‘Worried’ About Olympics...,2021-06-24,https://www.snopes.com/ap/2021/06/24/palace-ja...,"The delayed games open July 23, and the Paraly..."
5,Millions Skipped Church During Pandemic. Will ...,2021-06-29,https://www.snopes.com/ap/2021/06/29/millions-...,With millions of people having stayed home fro...
6,"Jim Bakker, His Church Settle Lawsuit Over COV...",2021-06-23,https://www.snopes.com/ap/2021/06/23/jim-bakke...,Jim Bakker and his southwestern Missouri churc...
7,Hong Kong to Ban Passenger Flights from UK to ...,2021-06-28,https://www.snopes.com/ap/2021/06/28/hong-kong...,"Under the classification, people who have stay..."
8,Falling Short: Why the White House Will Miss I...,2021-06-24,https://www.snopes.com/ap/2021/06/24/falling-s...,"With the July Fourth holiday approaching, the ..."
9,FAA Says US Airports Will Get $8 Billion in Pa...,2021-06-22,https://www.snopes.com/ap/2021/06/22/faa-says-...,The COVID-19 pandemic caused a steep drop in a...


In [ ]:
#Download
df.to_csv('data624.csv') 
files.download('data624.csv')